<a href="https://colab.research.google.com/gist/SK124/aa4b928c0349dc1e16879ef7b641e990/qbmg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" Imports neccesary for installing RDkit """

#!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!cp '/content/drive/My Drive/Miniconda3-latest-Linux-x86_64.sh' /content/
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/rdkit')
%cd /usr/local/lib/python3.7/site-packages/

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - ca-certificates==2020.1.1=0
    - certifi==2020.4.5.1=py37_0
    - cffi==1.14.0=py37he30daa8_1
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.1=py37h7b6447c_0
    - conda==4.8.3=py37_0
    - cryptography==2.9.2=py37h1ba5d50_0
    - idna==2.9=py_1
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20181209=hc058e9b_0
    - libffi==3.3=he6710b0_1
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1g=h7b6447c_0
    - pip==20.0.2=py37_3
    - pycosat==0.6.3=py37h7b6447c_0
    - pycparser==2.20=py_0
    - pyopenssl==19.1.0=py37_0
    - pysocks==1.7.1=py37_0
    - python==3.7.7=hcff3b4d_5
    - readline==8.0=h7b6447c_0
    - requests==2.23.0=py37_0
    - ruamel_yaml==0.15.87=py37h7b6447c_0
    - setuptools==

In [ ]:
%cd /usr/local/lib/python3.7/site-packages/
import re
import pandas as pd
import numpy as np
import random
import pickle
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from rdkit import Chem
from rdkit.Chem import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import RDConfig
from tqdm import tqdm

/usr/local/lib/python3.7/site-packages


In [ ]:
%cd /content/
path='/content/drive/My Drive/De NovoDrug/full_data_final.csv'
df=pd.read_csv(path)
df.head()

/content


,col
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...


In [ ]:
def csv2txt(dataframe):
   " Writes the CSV File to text file " 
   with open('drug.txt','w') as f:
       for i in range(len(dataframe)):
           text=dataframe['col'][i]
           f.write(text+'\n')

In [ ]:
csv2txt(df)

In [ ]:

def txt2list(fname):
  " Takes the molecules in text file to and store it in list "
  smiles = []
  with open(fname, 'r') as f:
     for line in f:
         smiles.append(line.split()[0])
  return smiles

In [ ]:
smiles=txt2list('drug.txt')

In [ ]:
len(smiles)

1000000

In [ ]:

def replace_halogen(string):
    """Regex to replace Br and Cl with single letters"""
    br = re.compile('Br')
    cl = re.compile('Cl')
    string = br.sub('R', string)
    string = cl.sub('L', string)

    return string


In [ ]:
def construct_vocabulary(smiles_list,fname):
    """Returns all the characters present in a text file.
       Uses regex to find characters/tokens of the format '[x]'."""
    add_chars = set()
    for i, smiles in enumerate(smiles_list):
        regex = '(\[[^\[\]]{1,10}])'

        """
        Regex Explaination : \ signifies a special sequence inside a specific set [] of characters for eg.([...CC[NH+]COCN...]) so \[...] , which can contain [NH+] like molecules inside it which is taken care by  the ... as explained in the following  
        To capture such [NH+],[C@@H],[...] molecules, ^ (start with operator) is used, which is to say such a token is going to start, inside which we give the \[\] to denote a specific sequence which will start with a '[' and end with ']' ie.[NH+] or [C@@H], here \ is used to say escape the inbetween enitities(C@@H,NH+) ie dont split them
        There can be numbers 1,2....10 to denote begining of rings within the sequence like CCCcc1n.... to capture these numbers {1,10} ie all numbers between 1 to 10, is used
        INPUT SMILE ----> 'CCC[C@@H][NH3+][O+][O-][OH+][P+]COCCCcc1nnn' 
        char_list = ['CCC','[C@@H]','[NH3+]','[O+]','[O-]','[OH+]','[P+]','COCCCcc1nnn']
         
        """
        smiles = replace_halogen(smiles)
        char_list = re.split(regex, smiles)
        for char in char_list:
        "There are characters like [NH+] which need to considered as single token rather than '[', 'N','H','+',']'"
            if char.startswith('['):
                add_chars.add(char)

            else:
            "chars inside charlist need to be split as well i.e. COCCCcc1nnn need to split into tokens like 'C','O','1','c','n' "
                chars = [unit for unit in char]
                [add_chars.add(unit) for unit in chars]

    print("Number of characters: {}".format(len(add_chars)))
    with open(fname, 'w') as f:
        for char in add_chars:
            f.write(char + "\n")
    return add_chars


In [ ]:
construct_vocabulary(smiles,'vocab.txt')

Number of characters: 58


{'#',
 '(',
 ')',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '=',
 'C',
 'F',
 'I',
 'L',
 'N',
 'O',
 'P',
 'R',
 'S',
 '[C@@H]',
 '[C@@]',
 '[C@H]',
 '[C@]',
 '[CH-]',
 '[CH2-]',
 '[N+]',
 '[N-]',
 '[NH+]',
 '[NH-]',
 '[NH2+]',
 '[NH3+]',
 '[O+]',
 '[O-]',
 '[OH+]',
 '[P+]',
 '[P@@H]',
 '[P@@]',
 '[P@]',
 '[PH2]',
 '[S+]',
 '[S-]',
 '[S@@]',
 '[S@]',
 '[SH+]',
 '[n+]',
 '[n-]',
 '[nH+]',
 '[nH]',
 '[o+]',
 '[s+]',
 '\\',
 'c',
 'n',
 'o',
 's'}

In [ ]:
class Vocabulary(object):
    """A class for handling encoding/decoding from SMILES to an array of indices"""
    def __init__(self, init_from_file=None, max_length=100):
        "E and G are used to signify ending and begining of seqeunce"
        self.special_tokens = ['E', 'G']
        "Other tokens frmo SMILES grammer need to be added"
        self.additional_chars = set()
        self.chars = self.special_tokens
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars)))) 
       # self.vocab = dict((c, i+1) for i, c in enumerate(self.chars))
        self.reversed_vocab = {v: k for k, v in self.vocab.items()}
       #self.reversed_vocab = {v+1: k for k, v in self.vocab.items()} 
        self.max_length = max_length
        if init_from_file: 
        " vocab.txt file which contains all the tokens is given as a input which initiates the vocabulary for the language model"   
          self.init_from_file(init_from_file)

    def encode(self, char_list):
        """Takes a list of characters (eg '[C@@H]') and encodes to array of indices"""
        smiles_matrix = np.zeros(len(char_list), dtype=np.float32)
        for i, char in enumerate(char_list):
          "2D Matrix : All the tokens are converted from SMILES to integers"
            smiles_matrix[i] = self.vocab[char]
        return smiles_matrix

    def decode(self, matrix):
        """Takes an array of indices and returns the corresponding SMILES"""
        chars = []
        for i in matrix:
            "If E token which is the End token is reached, the process terminates"
            if i == self.vocab['E']: break
            chars.append(self.reversed_vocab[i])
        smiles = "".join(chars)
        smiles = smiles.replace("L", "Cl").replace("R", "Br")
        return smiles

    def tokenize(self, smiles):
        """Takes a SMILES and return a list of characters/tokens"""
        regex = '(\[[^\[\]]{1,10}\])'
        smiles = replace_halogen(smiles)
        char_list = re.split(regex, smiles)
        tokenized = []
        "SMILES Sequence needs to be converted to tokens for preprocessing, this is the same code we had in Vocabulary class"
        for char in char_list:
            if char.startswith('['):
                tokenized.append(char)
            else:
                chars = [unit for unit in char]
                [tokenized.append(unit) for unit in chars]
        "E token is added to signify the ending of a token"
        tokenized.append('E')
        return tokenized

    def add_characters(self, chars):
        """Adds characters to the vocabulary"""
        for char in chars:
            self.additional_chars.add(char)
        char_list = list(self.additional_chars)
        char_list.sort()
        self.chars = char_list + self.special_tokens
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.reversed_vocab = {v: k for k, v in self.vocab.items()}


    def init_from_file(self, file):
        """Takes a file containing \n separated characters to initialize the vocabulary"""
        with open(file, 'r') as f:
          "All characters from vocab.txt file which has all the unique tokens is taken and added to the Vocabukary of the model"  
            chars = f.read().split()
        self.add_characters(chars)

    def __len__(self):
        return len(self.chars)

    def __str__(self):
        return "Vocabulary containing {} tokens: {}".format(len(self), self.chars)

class MolData(Dataset):
    """Custom PyTorch Dataset that takes a file containing SMILES.
        Args:
                fname : path to a file containing \n separated SMILES.
                voc   : a Vocabulary instance
        Returns:
                A custom PyTorch dataset for training the Prior.
    """
    def __init__(self, fname, voc):
        self.voc = voc
        self.smiles = []
        with open(fname, 'r') as f:
            for line in f:
                self.smiles.append(line.split()[0])

    def __getitem__(self, i): 
        mol = self.smiles[i]
        tokenized = self.voc.tokenize(mol)
        encoded = self.voc.encode(tokenized)
        return Variable(encoded)

    def __len__(self):
        return len(self.smiles)

    def __str__(self):
        return "Dataset containing {} structures.".format(len(self))

    @classmethod
    def collate_fn(cls, arr):
        """Function to take a list of encoded sequences and turn them into a batch"""
        max_length = max([seq.size(0) for seq in arr])
        """ For every batch lets say 64, 64 sequences will be taken an the maximum length of the sequence from that batch woul be max_length 
        and the shorter seqeunces will be padded till the max_length before passing it to the GRU """
        collated_arr = Variable(torch.zeros(len(arr), max_length))
        for i, seq in enumerate(arr):
            collated_arr[i, :seq.size(0)] = seq
        return collated_arr


def Variable(tensor):
    """Wrapper for torch.autograd.Variable that also accepts
       numpy arrays directly and automatically assigns it to
       the GPU."""
    if isinstance(tensor, np.ndarray):
        tensor = torch.from_numpy(tensor)
    if torch.cuda.is_available():
        return torch.autograd.Variable(tensor).cuda()
    return torch.autograd.Variable(tensor)

def decrease_learning_rate(optimizer, decrease_by=0.01):
    """Multiplies the learning rate of the optimizer by 1 - decrease_by"""
    for param_group in optimizer.param_groups:
        param_group['lr'] *= (1 - decrease_by)

In [ ]:
class GRU(nn.Module):
    """ Implements a three layer GRU cell including an embedding layer
       and an output linear layer back to the size of the vocabulary"""
    def __init__(self, voc_size):
        super(GRU, self).__init__()
        self.embedding = nn.Embedding(voc_size, 128)
        self.gru_1 = nn.GRUCell(128, 512)
        self.gru_2 = nn.GRUCell(512, 512)
        self.gru_3 = nn.GRUCell(512, 512)
        self.linear = nn.Linear(512, voc_size)

    def forward(self, x, h):
        x = self.embedding(x)
        h_out = Variable(torch.zeros(h.size()))
        x = h_out[0] = self.gru_1(x, h[0])
        x = h_out[1] = self.gru_2(x, h[1])
        x = h_out[2] = self.gru_3(x, h[2])
        x = self.linear(x)
        return x, h_out

    def init_h(self, batch_size):
        # Initial cell state is zero
        return Variable(torch.zeros(3, batch_size, 512))

class RNN():
    """Implements the Prior and Agent RNN. Needs a Vocabulary instance in
    order to determine size of the vocabulary and index of the END token"""
    def __init__(self, voc):
        self.rnn = GRU(voc.vocab_size)
        if torch.cuda.is_available():
            self.rnn.cuda()
        self.voc = voc

    def likelihood(self, target):
        """
            Retrieves the likelihood of a given sequence
            Args:
                target: (batch_size * sequence_lenght) A batch of sequences
            Outputs:
                log_probs : (batch_size) Log likelihood for each example*
               
        """
        batch_size, seq_length = target.size()
        start_token = Variable(torch.zeros(batch_size, 1).long())
        start_token[:] = self.voc.vocab['G'] 
        x = torch.cat((start_token, target[:, :-1]), 1) # G (beginning token) is added to each sequence in the batch.
        h = self.rnn.init_h(batch_size) # hidden state is also initalized as per batch size
        log_probs = Variable(torch.zeros(batch_size)) #Variable to store log_probabilities 

        for step in range(seq_length):
          """ for every step from 1 to sequnce length, we pass the GRU, Seqeunce based on the timestep and hidden state and get logits and 
          hidden state after taking that step
          logits is a vector of 'D' dimension where D = voc_size with scores on what should be the next token, which is converted to 
          log_probs using log_softmax and sent to the NLL Loss function which compares the next predicted word by RNN and Ground Truth and calculates 
          the loss per sequence per timestep which is accumalted under log_probs and returns total loss after end time step """
            logits, h = self.rnn(x[:, step], h)
            log_prob = F.log_softmax(logits)
            log_probs += NLLLoss(log_prob, target[:, step])
        return log_probs

    def sample(self, batch_size, max_length=150):
        """
            Sample a batch of sequences
            Args:
                batch_size : Number of sequences to sample 
                max_length:  Maximum length of the sequences
            Outputs:
            seqs: (batch_size, seq_length) The sampled sequences.
            log_probs : (batch_size) Log likelihood for each sequence.
            
        """
        start_token = Variable(torch.zeros(batch_size).long())
        start_token[:] = self.voc.vocab['G']
        h = self.rnn.init_h(batch_size)
        x = start_token

        sequences = []
        log_probs = Variable(torch.zeros(batch_size))
        finished = torch.zeros(batch_size).byte()
        if torch.cuda.is_available():
            finished = finished.cuda()

        for step in range(max_length):
          """ After language model learns the semantic syntax, we need to check its perfomance which is why we sample from it.
          The code here is exactly similar to the code in likeilhood function """ 
            logits, h = self.rnn(x, h)
            prob = F.softmax(logits)
            log_prob = F.log_softmax(logits)
            """ torch.multinomial returns a tensor where each row contains 1 index sampled from the multinomial probability distribution 
            located in the corresponding row of tensor input.
            """
            x = torch.multinomial(prob,1).view(-1)
            sequences.append(x.view(-1, 1))
            log_probs +=  NLLLoss(log_prob, x)

            x = Variable(x.data)
            """ Sampling terminates when E token is spit out of GRU """
            EOS_sampled = (x == self.voc.vocab['E']).data
            finished = torch.ge(finished + EOS_sampled, 1)
            if torch.prod(finished) == 1: break

        sequences = torch.cat(sequences, 1)
        return sequences.data, log_probs

def NLLLoss(inputs, targets):
    """
        Custom Negative Log Likelihood loss that returns loss per example,
        rather than for the entire batch.
        Args:
            inputs : (batch_size, num_classes) *Log probabilities of each class*
            targets: (batch_size) *Target class index*
        Outputs:
            loss : (batch_size) *Loss for each example*
    """

    if torch.cuda.is_available():
        target_expanded = torch.zeros(inputs.size()).cuda()
    else:
        target_expanded = torch.zeros(inputs.size())

    target_expanded.scatter_(1, targets.contiguous().view(-1, 1).data, 1.0)
    loss = Variable(target_expanded) * inputs
    loss = torch.sum(loss, 1)
    return loss

" Using the log-softmax will punish bigger mistakes in likelihood space higher."

In [ ]:
def pretrain(restore_from = None):
    """Trains the Prior RNN"""

    voc = Vocabulary(init_from_file="vocab.txt")
    moldata = MolData("drug.txt", voc)
    data = DataLoader(moldata, batch_size=128, shuffle=True, drop_last=True,
                      collate_fn=MolData.collate_fn)
    
    Prior = RNN(voc)
    
    # Can restore from a saved RNN
    if restore_from:
        Prior.rnn.load_state_dict(torch.load(restore_from))

    optimizer = torch.optim.Adam(Prior.rnn.parameters(), lr = 0.001)
    for epoch in range(1, 51):
        # When training on a few million compounds, this model converges
        # in a few of epochs or even faster. If model sized is increased
        # its probably a good idea to check loss against an external set of
        # validation SMILES to make sure we dont overfit too much.
        for step, batch in tqdm(enumerate(data), total=len(data)):

            # Sample from DataLoader
            seqs = batch.long()

            # Calculate loss
            log_p= Prior.likelihood(seqs)
            loss = - log_p.mean()

            # Calculate gradients and take a step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Every 500 steps we decrease learning rate and print some information
            if step % 500 == 0 and step != 0:
                #decrease_learning_rate(optimizer, decrease_by=0.003)
                decrease_learning_rate(optimizer, decrease_by=0.001)
            if step % 30 == 0 and step != 0:
                tqdm.write("*" * 50)
                tqdm.write("Epoch {:3d}   step {:3d}    loss: {:5.2f}\n".format(epoch, step, loss.data))
                seqs, likelihood = Prior.sample(100)
                valid = 0
                f = open('test_output.txt', 'a')
                for i, seq in enumerate(seqs.cpu().numpy()):
                        smile = voc.decode(seq)
                        if Chem.MolFromSmiles(smile):
                           valid += 1
                           f.write(smile + "\n")
                           if i < 10:
                             tqdm.write(smile)
                f.close()
                tqdm.write("\n{:>4.1f}% valid SMILES".format(100 * valid / len(seqs)))
                tqdm.write("*" * 50 + "\n")               
        # Save the Prior
        torch.save(Prior.rnn.state_dict(), "50_epoch.ckpt")

if __name__ == "__main__":
#     smiles_file = sys.argv[1]
    smiles_file = '/content/drug.txt'
    print("Reading smiles...")
    smiles_list = smiles
    print("Constructing vocabulary...")
    voc_chars = construct_vocabulary(smiles_list, "vocab.txt")
   # ds.write_smiles_to_file(smiles_list, "./mols_filtered.smi")

Reading smiles...
Constructing vocabulary...
Number of characters: 58


In [ ]:
pretrain(restore_from = None)




  0%|          | 0/781 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



  0%|          | 1/781 [00:00<02:27,  5.28it/s]

59





  0%|          | 2/781 [00:00<02:24,  5.38it/s]

65
60





  0%|          | 3/781 [00:00<02:16,  5.68it/s]


  1%|          | 4/781 [00:00<02:12,  5.87it/s]

...









  7%|▋         | 53/781 [34:30<02:00,  6.03it/s]


 96%|██████▉   | 540/781 [01:41<00:43,  5.54it/s]







  7%|▋         | 53/781 [34:30<02:00,  6.03it/s]


 96%|██████▉   | 540/781 [01:41<00:43,  5.54it/s]







  7%|▋         | 53/781 [34:30<02:00,  6.03it/s]


 96%|██████▉   | 540/781 [01:41<00:43,  5.54it/s]







  7%|▋         | 53/781 [34:30<02:00,  6.03it/s]


 97%|██████▉   | 540/781 [01:41<00:43,  5.54it/s]







  7%|▋         | 53/781 [34:30<02:00,  6.03it/s]


 99%|██████▉   | 540/781 [01:41<00:43,  5.54it/s]







  7%|▋         | 53/781 [34:30<02:00,  6.03it/s]


 69%|██████▉   | 540/781 [01:41<00:43,  5.54it/s]







  7%|▋         | 53/781 [34:30<02:00,  6.03it/s]


 99%|██████▉   | 540/781 [01:41<00:43,  5.54it/s]

  4%|▍         | 30/781 [03:20<02:07,  5.87it/s]


 99%|██████▉   | 541/781 [01:41<01:10,  3.41it/s]

Cc1ccc(NC(=O)c2c(NC(=O)CCC(C)C)oc(C)c2)CCO1
CSCc1nc(N(Cc2ccccc2)S(C)(=O)=O)cc(C)c1OC
CC[NH2+][C@H]1CCC[C@H](CC)[C@@H](C)O1
COC(=O)[C@H]1CCCN1C(=O)NCc1con(C)c1=O
CC[S@](=O)C[C@H](C)Cn1c2ccccc2nc1[C@@H]1CC[C@H](N(C)C)C1

90.0% valid SMILES
**************************************************

65





 99%|██████▉   | 542/781 [01:41<01:10,  3.39it/s]


 100%|██████▉   | 543/781 [01:42<01:01,  3.85it/s]

63
63


KeyboardInterrupt: ignored

In [ ]:
mv /content/50_epoch.ckpt '/content/drive/My Drive/De NovoDrug'